# **Machine Learning para Business Intelligence** 
#### Profesor: Santiago Neira


## Clase 3. Introducción a regresiones

In [ ]:
##Paquetes que vamos a usar 
##!pip install pandas numpy scikit-learn seaborn statsmodels

In [ ]:
# Importamos las librerías de la clase
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
import seaborn as sns

Para la clase de hoy, continuaremos con los datos de calidad de vinos

In [ ]:
calidad_vinos = pd.read_csv('../Datos/wine-quality.csv')

Nuestro objetivo será ahora predecir su calidad.

In [ ]:
calidad_vinos.head()

### 0. Manipulación de datos y Feature Engineering

Quisieramos calcular otra información a partir de la que tenemos. En particular
- El **tipo** de vino basado en una lista: *Cabernet Sauvignon, Chardonnay, Pinot Noir*,...
- El **año** del vino basado en su título.

In [ ]:
tipo_vino = ['Cabernet Sauvignon','Chardonnay','Pinot Noir','Sparkling','Sauvignon Blanc','Merlot',\
              'Malbec','Prosecco','Rosé','Syrah','Zynfandel','Tempranillo','Moscato','Riesling','Pinot Gris']

Creamos una función que nos permita encontrar un tipo de vino a partir de esta lista.

In [ ]:
def encontrar_tipo_vino(text: str):
    tipo = 'Otro'
    for k in tipo_vino:
        if k.lower() in text.lower():
            tipo = k
            break
    return tipo

In [ ]:
encontrar_tipo_vino('Basel Cellars 2011 Merlot (Columbia Valley (WA))')

In [ ]:
encontrar_tipo_vino('DFJ Vinhos 2011 Vega Red (Douro)')

In [ ]:
encontrar_tipo_vino('Beaulieu Vineyard 2013 Maestro Collection Cabernet Sauvignon-Syrah (Napa Valley)')

Ahora, aplicamos esta función a todos los títulos.

In [ ]:
calidad_vinos['tipo vino'] = calidad_vinos.title.apply(encontrar_tipo_vino)

Es fácil confundirse con la columna `type`, que en realidad hace referencia al color.

In [ ]:
calidad_vinos = calidad_vinos.rename(columns={'type':'color'})

Quisiéramos sacar también el año en el que salió un vino. Para esto utilizamos un paquete conocido como `RegEx` basado en expresiones regulares. Estas las cubriremos más adelante en el curso, por ahora ¡observen la magia!

In [ ]:
calidad_vinos['title'].apply(lambda s: re.findall(r'\d{4}', s)[0] if len(re.findall(r'\d{4}', s)) > 0 else np.nan)

In [ ]:
calidad_vinos['año'] = pd.to_numeric(calidad_vinos['title'].apply(lambda s: re.findall(r'\d{4}', s)[0] if len(re.findall(r'\d{4}', s)) > 0 else np.nan))

In [ ]:
calidad_vinos.head()

Las características inherentes de los vinos pueden estar plenamente caracterizadas por su tipo y el color! Podemos pensar en imputar los datos que nos faltan con base en esto!

In [ ]:
imputacion_color_tipo = calidad_vinos.groupby(['color','tipo vino'])[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','pH','sulphates','año']].mean()
imputacion_color_tipo.head()

In [ ]:
imputacion_color_tipo = imputacion_color_tipo.reset_index()
imputacion_color_tipo['llave'] = imputacion_color_tipo['color']+'-'+imputacion_color_tipo['tipo vino']

In [ ]:
imputacion_color_tipo = imputacion_color_tipo.drop(columns=['color','tipo vino']).set_index('llave')
imputacion_color_tipo.head()

In [ ]:
calidad_vinos['llave'] = calidad_vinos['color']+'-'+calidad_vinos['tipo vino']
calidad_vinos.head()

In [ ]:
calidad_vinos['llave'].map(imputacion_color_tipo['fixed acidity'])

In [ ]:
calidad_vinos['fixed acidity'].fillna(calidad_vinos['llave'].map(imputacion_color_tipo['fixed acidity']))

In [ ]:
vars_imputar = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','pH','sulphates','año']
for columna_seleccionada in vars_imputar:
    calidad_vinos[columna_seleccionada] = calidad_vinos[columna_seleccionada].fillna(calidad_vinos['llave'].map(imputacion_color_tipo[columna_seleccionada]))

In [ ]:
calidad_vinos.isna().sum()

In [ ]:
calidad_vinos.sample(5)

In [ ]:
calidad_vinos.drop(columns='llave',inplace=True)

In [ ]:
calidad_vinos['año'] = calidad_vinos['año'].astype(int)

In [ ]:
calidad_vinos.head()

### Visualicemos nuestro datos

In [ ]:
calidad_vinos['quality'] = calidad_vinos['quality'].astype(float)
calidad_vinos['fixed acidity'] = calidad_vinos['fixed acidity'].astype(float)
calidad_vinos['año'] = calidad_vinos['año'].astype(float)

In [ ]:
sns.pairplot(data=calidad_vinos[['fixed acidity','volatile acidity','citric acid','total sulfur dioxide','residual sugar','sulphates','alcohol','chlorides','density','pH','año','color','quality']],hue='color')

In [ ]:
variables_numericas = ['fixed acidity','volatile acidity','citric acid','residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol','año']

In [ ]:
variables_categoricas=['color','tipo vino']

In [ ]:
X = calidad_vinos[variables_numericas+variables_categoricas]
y = calidad_vinos['quality']

In [ ]:
X = pd.get_dummies(X,columns=variables_categoricas,drop_first=True)

In [ ]:
X.isna().mean()

##  Veamos nuestro primer modelo!! 

In [ ]:
###Train - test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)

In [ ]:
print(X_train.shape)

In [ ]:
X_test.shape

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
## Corramos nuestra primera regresión -- útil para quienes quieren ver relevancia estadística
import statsmodels.api as sm
X_train_adj = sm.add_constant(X_train)
X_train_adj = X_train_adj.astype(float)
model = sm.OLS(endog=y_train,exog=X_train_adj)
results = model.fit()

In [ ]:
results.summary()

In [ ]:
## Hay alta multicolinealidad
correlation=X.corr()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8)) 

# Create the heatmap
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.1f', cbar=True, 
            annot_kws={'size': 7},  
            linewidths=0.5,  
            linecolor='gray', 
            square=True) 

# Add title
plt.title("Correlation Matrix", fontsize=16)

# Adjust the layout to avoid clipping of labels
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
mask = correlation.where(abs(correlation) > 0.6)
filtered_pairs = mask.stack().reset_index()
filtered_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
filtered_pairs = filtered_pairs[filtered_pairs['Feature 1'] != filtered_pairs['Feature 2']]
print(filtered_pairs)

In [ ]:
from sklearn.preprocessing import StandardScaler
## Re-definir las variables a utilizar, y correr nuevamente el ejercicio
variables_numericas = ['fixed acidity','volatile acidity','citric acid','residual sugar', 'chlorides', 'total sulfur dioxide', 'pH', 'sulphates', 'alcohol','año']
variables_categoricas=['tipo vino']

estandarizador = StandardScaler()
estandarizador.fit(calidad_vinos.loc[:,variables_numericas])
calidad_vinos.loc[:,variables_numericas] = estandarizador.transform(calidad_vinos.loc[:,variables_numericas])


X = calidad_vinos[variables_numericas+variables_categoricas]
y = calidad_vinos['quality']
X = pd.get_dummies(X,columns=variables_categoricas,drop_first=True)


##Train test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)
X_train_adj = sm.add_constant(X_train)
X_train_adj = X_train_adj.astype(float)
##Modelo
model = sm.OLS(endog=y_train,exog=X_train_adj)
results = model.fit()
results.summary()

## Métricas de evaluación

In [ ]:
### Ahora utilicemos el paquete que nos permitirá hacer otro tipo de análisis
from sklearn.linear_model import LinearRegression

In [ ]:
modelo_regresion = LinearRegression()
modelo_regresion.fit(X_train,y_train)

In [ ]:
modelo_regresion.predict(X_test)

In [ ]:
final = X_test.copy()
final['prediccion'] = modelo_regresion.predict(X_test)
final['calidad real'] = y_test

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,r2_score

In [ ]:
mean_absolute_error(y_true=y_train,y_pred=modelo_regresion.predict(X_train))

In [ ]:
mean_absolute_error(y_true=y_test,y_pred=modelo_regresion.predict(X_test))

In [ ]:
mean_absolute_percentage_error(y_true=y_train,y_pred=modelo_regresion.predict(X_train))

In [ ]:
mean_absolute_percentage_error(y_true=y_test,y_pred=modelo_regresion.predict(X_test))

In [ ]:
r2_score(y_test,modelo_regresion.predict(X_test))

In [ ]:
r2_score(y_train,modelo_regresion.predict(X_train))

## Probemos otra especificación - Utilicemos lo recogido del modelo original

In [ ]:
calidad_vinos.loc[:,variables_numericas] = estandarizador.inverse_transform(calidad_vinos.loc[:,variables_numericas])

In [ ]:
## Probemos otra especificación -- utilizemos lo recogido del modelo original
variables_grado_poly = ['volatile acidity','residual sugar', 'total sulfur dioxide',  'sulphates', 'alcohol']

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
trans_polinomial = PolynomialFeatures(degree=2)
trans_polinomial.fit(calidad_vinos.loc[:,variables_grado_poly])

In [ ]:
pd.DataFrame(trans_polinomial.transform(calidad_vinos.loc[:,variables_grado_poly]),
             columns=trans_polinomial.get_feature_names_out(input_features=variables_grado_poly))

In [ ]:
calidad_vinos_poly = pd.concat([calidad_vinos.drop(columns=variables_grado_poly),pd.DataFrame(trans_polinomial.transform(calidad_vinos.loc[:,variables_grado_poly]),
             columns=trans_polinomial.get_feature_names_out(input_features=variables_grado_poly))],axis=1)

In [ ]:
calidad_vinos_poly.columns

In [ ]:
variables_numericas_poly = list(set(variables_numericas + list(trans_polinomial.get_feature_names_out(input_features=variables_grado_poly))))
variables_categoricas = ['color']

In [ ]:
estandarizador = StandardScaler()
estandarizador.fit(calidad_vinos_poly.loc[:,variables_numericas_poly])
calidad_vinos_poly.loc[:,variables_numericas_poly] = estandarizador.transform(calidad_vinos_poly.loc[:,variables_numericas_poly])

In [ ]:
X=calidad_vinos_poly[variables_numericas_poly+variables_categoricas]
X = pd.get_dummies(X,columns=variables_categoricas,drop_first=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33)

In [ ]:
modelo_regresion = LinearRegression()
modelo_regresion.fit(X_train,y_train)

In [ ]:
mean_absolute_error(y_true=y_train,y_pred=modelo_regresion.predict(X_train))

In [ ]:
mean_absolute_error(y_true=y_test,y_pred=modelo_regresion.predict(X_test))

In [ ]:
mean_absolute_percentage_error(y_true=y_train,y_pred=modelo_regresion.predict(X_train))

In [ ]:
mean_absolute_percentage_error(y_true=y_test,y_pred=modelo_regresion.predict(X_test))

In [ ]:
r2_score(y_test,modelo_regresion.predict(X_test))

In [ ]:
r2_score(y_train,modelo_regresion.predict(X_train))

In [ ]:
pd.DataFrame(modelo_regresion.coef_,index=modelo_regresion.feature_names_in_).rename(columns={0:'coef'}).sort_values(by='coef',ascending=False).head()

## Regularización

In [ ]:
X = calidad_vinos_poly[variables_numericas_poly+variables_categoricas]
y = calidad_vinos_poly['quality']

### Regresión Ridge - Regularización L2

In [ ]:
from sklearn.linear_model import Ridge
regresion_ridge = Ridge(alpha=1)
regresion_ridge.fit(X_train,y_train)

In [ ]:
mean_absolute_percentage_error(y_true=y_test,y_pred=np.round(regresion_ridge.predict(X_test)))

In [ ]:
pd.DataFrame(regresion_ridge.coef_,index=regresion_ridge.feature_names_in_).rename(columns={0:'coef'}).sort_values(by='coef',ascending=False).head()

In [ ]:
regresion_ridge = Ridge(alpha=2)
regresion_ridge.fit(X_train,y_train)

In [ ]:
mean_absolute_percentage_error(y_true=y_test,y_pred=np.round(regresion_ridge.predict(X_test)))

In [ ]:
pd.DataFrame(regresion_ridge.coef_,index=regresion_ridge.feature_names_in_).rename(columns={0:'coef'}).sort_values(by='coef',ascending=False).head()

### Regresión Lasso - Regularización L1

In [ ]:
from sklearn.linear_model import Lasso
regresion_lasso = Lasso(alpha=1)
regresion_lasso.fit(X_train,y_train)
mean_absolute_percentage_error(y_true=y_test,y_pred=np.round(regresion_lasso.predict(X_test)))

In [ ]:
regresion_lasso = Lasso(alpha=0.01)
regresion_lasso.fit(X_train,y_train)
mean_absolute_percentage_error(y_true=y_test,y_pred=np.round(regresion_lasso.predict(X_test)))

In [ ]:
pd.DataFrame(regresion_lasso.coef_,index=regresion_lasso.feature_names_in_).rename(columns={0:'coef'}).sort_values(by='coef',ascending=False).head(50)

### Validación Cruzada

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

scores = cross_val_score(regresion_lasso,X_test,y_test,cv=5,scoring='neg_mean_absolute_percentage_error')

In [ ]:
scores

### Búsqueda de hiperparámetros para Lasso

In [ ]:
from sklearn.model_selection import GridSearchCV

busqueda_cuadricula = GridSearchCV(Lasso(),param_grid={'alpha':[0.0001,0.001,0.01,0.1,1,10]},scoring='neg_mean_absolute_percentage_error')
busqueda_cuadricula.fit(X_train,y_train)

In [ ]:
pd.DataFrame(busqueda_cuadricula.cv_results_)

In [ ]:
pd.DataFrame(busqueda_cuadricula.cv_results_).plot(x='param_alpha',y='mean_test_score',xlabel=r'Párametro $\alpha$',ylabel='Score promedio')

In [ ]:
np.logspace(-4,-2,20)

In [ ]:
from sklearn.model_selection import GridSearchCV

busqueda_cuadricula = GridSearchCV(Lasso(),param_grid={'alpha':np.logspace(-5,-2,20)},scoring='neg_mean_absolute_percentage_error')
busqueda_cuadricula.fit(X_train,y_train)

In [ ]:
_ = pd.DataFrame(busqueda_cuadricula.cv_results_).plot(x='param_alpha',y='mean_test_score',xlabel=r'Párametro $\alpha$',ylabel='Score promedio')

In [ ]:
busqueda_cuadricula.best_estimator_

In [ ]:
busqueda_cuadricula.best_estimator_.alpha

In [ ]:
pd.Series(busqueda_cuadricula.best_estimator_.coef_,index=busqueda_cuadricula.best_estimator_.feature_names_in_).sort_values()

In [ ]:
### Volvamos a correr el modelo con esto y comparemos

In [ ]:
mean_absolute_percentage_error(y_test,busqueda_cuadricula.predict(X_test))
##En nuestra construcción original teníamos 0.10150062728065513

 Tarea - Un pedazo que no estamos corriendo hace parte de la selección de variables. Escoja aquellas variables que son no-negativas y corra el `pipeline` completo de selección. Llega a nuevos resultados?

### Búsqueda de hiperparámetros para Ridge

In [ ]:
busqueda_cuadricula_ridge = GridSearchCV(Ridge(),param_grid={'alpha':np.logspace(-5,1,20)},scoring='neg_mean_absolute_percentage_error')
busqueda_cuadricula_ridge.fit(X_train,y_train)

In [ ]:
_ = pd.DataFrame(busqueda_cuadricula_ridge.cv_results_).plot(x='param_alpha',y='mean_test_score',xlabel=r'Párametro $\alpha$',ylabel='Score promedio')

In [ ]:
busqueda_cuadricula_ridge.predict(X_test)

In [ ]:
mean_absolute_percentage_error(y_test,busqueda_cuadricula_ridge.predict(X_test))

In [ ]:
busqueda_cuadricula_ridge.best_estimator_

In [ ]:
pd.Series(busqueda_cuadricula_ridge.best_estimator_.coef_,index=busqueda_cuadricula_ridge.best_estimator_.feature_names_in_).sort_values()